In [18]:
import functools
import parse

def parse_line(s):
    r = parse.parse("Valve {valve} has flow rate={flow:d}; {} to {} {next}", s).named
    r["next"] = r["next"].split(", ")
    return (r["valve"], {"flow":r["flow"], "next":r["next"]})

input = dict([parse_line(l) for l in sorted(open("inputs/16-input.txt").read().splitlines())])

input

@functools.lru_cache(maxsize=None)
def optimise1(opened, mins, v):
    if mins <= 0:
        return 0
    best = 0
    s = input[v]
    
    # Choose next valve
    for valve in s["next"]:
        best_next = optimise1(opened, mins-1, valve)
        best = max(best, best_next)

    # If current valve closed, choose to open or move to next valve
    if v not in opened and s["flow"] > 0 and mins > 0:
        opened = set(opened)
        opened.add(v)
        mins -= 1
        new_sum = mins * s["flow"]

        for valve in s["next"]:
            best_next = new_sum + optimise1(frozenset(opened), mins-1, valve)
            best = max(best, best_next)

    return best

optimise1(frozenset(), 30, "AA")

@functools.lru_cache(maxsize=None)
def optimise2(opened, mins, v):
    
    # At end, repeat for second player
    if mins <= 0:
        return optimise1(opened, 26, "AA")
    best = 0
    s = input[v]
    
    # Choose next valve
    for valve in s["next"]:
        best_next = optimise2(opened, mins-1, valve)
        best = max(best, best_next)

    # If current valve closed, choose to open or move to next valve
    if v not in opened and s["flow"] > 0 and mins > 0:
        opened = set(opened)
        opened.add(v)
        mins -= 1
        new_sum = mins * s["flow"]

        for valve in s["next"]:
            best_next = new_sum + optimise2(frozenset(opened), mins-1, valve)
            best = max(best, best_next)

    return best

optimise2(frozenset(), 26, "AA")

2292